<a href="https://colab.research.google.com/github/ReynaQuita/NLP/blob/main/Translation_t5_wo_fine_tuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets transformers sacrebleu

     |████████████████████████████████| 225kB 8.4MB/s 
     |████████████████████████████████| 2.3MB 14.2MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 245kB 47.7MB/s 
     |████████████████████████████████| 112kB 47.0MB/s 
     |████████████████████████████████| 901kB 43.0MB/s 
     |████████████████████████████████| 3.3MB 42.8MB/s 
ERROR: transformers 4.6.1 has requirement huggingface-hub==0.0.8, but you'll have huggingface-hub 0.0.9 which is incompatible.


In [ ]:
model_checkpoint = "t5-small"

In [ ]:
# from datasets import load_dataset, load_metric

# train_datasets = load_dataset("wmt16", "ro-en", split="train[:1%]")
# val_datasets = load_dataset("wmt16", "ro-en", split="validation[:1%]")


In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
import datasets

test_data = datasets.load_dataset("wmt16", "ro-en", split="test[:5%]")
metric = datasets.load_metric("sacrebleu")
len(test_data)

Reusing dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a)


100

In [ ]:
def generate_translation(batch):
    # cut off at BERT max length 512

    english = [ex["en"] for ex in batch["translation"]]
    # romanian = [ex["ro"] for ex in batch["translation"]]

    inputs = tokenizer(english, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    # input_ids = inputs.input_ids.to("cuda")
    # attention_mask = inputs.attention_mask.to("cuda")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    outputs = model.generate(input_ids, attention_mask=attention_mask)

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_translation"] = output_str


    return batch

In [ ]:
batch_size = 4  # change to 64 for full evaluation

results = test_data.map(generate_translation, batched=True, batch_size=batch_size)

In [ ]:
bleu_score = 0

for i in range(len(results)):
  ref = results["translation"][i]["ro"]
  pred = results["pred_translation"][i]

  bleu_metric = metric.compute(predictions=[pred], references=[[ref]])
  bleu_score  += bleu_metric["score"]


In [ ]:
print('Bleu Score: {} (scale 0-100)'.format(bleu_score/len(results)))

Bleu Score: 3.1285407718530456 (scale 0-100)
